In [45]:
import oyaml as yaml
import os
from os import path

class Vault():
    def __init__(self, root_path):
        self.ROOT_PATH = root_path

    def make_change(self, change):
        self.apply_change(change, save=True)

    def preview_change(self, change):
        self.apply_change(change, save=False)

    def apply_change(self, change, save):
        for dirpath, _, filenames in os.walk(self.ROOT_PATH):
            for filename in filenames:
                if filename.endswith('.md'):
                    file_path = path.join(dirpath, filename)
                    with open(file_path, 'r', encoding='utf-8') as file:
                        content = file.readlines()

                    frontmatter, content = Vault.extract_frontmatter(content)
                    new_frontmatter, new_content = change(frontmatter, content)

                    if save:
                        self.save_file(file_path, new_frontmatter, new_content)
                    else:
                        self.print_preview(new_frontmatter, new_content)

    def extract_frontmatter(content):
        if not content:
            return ({}, [])

        frontmatter_lines = []
        content_lines = []
        
        # does the file have a frontmatter?
        if content[0].strip() == '---':
            valid_frontmatter = False
            for line in content[1:]:
                if line.strip() == '---':
                    valid_frontmatter = True
                    break

                frontmatter_lines.append(line)
            
            if not valid_frontmatter:
                raise ValueError("Invalid frontmatter definition!")
            
            frontmatter = yaml.load(''.join(frontmatter_lines), Loader=yaml.Loader) if frontmatter_lines else {}
        
            content_lines = content[len(frontmatter_lines) + 2:]
        # no frontmatter, so we just return the content
        else:
            content_lines = content
            frontmatter = {}


        return (frontmatter, content_lines)

    def save_file(self, file_path, frontmatter, content):
        with open(file_path, 'w', encoding='utf-8') as file:
            if frontmatter:
                file.write('---\n')
                file.write(yaml.dump(frontmatter, default_flow_style=False))
                file.write('---\n')
            file.writelines(content)

    def print_preview(self, frontmatter, content):
        print('---')
        print(yaml.dump(frontmatter, default_flow_style=False), end='')
        print('---')
        print(''.join(content))


In [5]:
sample_content = """---
type: "note"
authors:
  - "cedric"
  - "The Universe"
  - "Keychron"
created: "2024-10-17"
---
# Hello World!

This is my story. It is short, and abru"""

sample_lines = sample_content.split("\n")

frontmatter, content = Vault.extract_frontmatter(sample_lines)

NameError: name 'Vault' is not defined

In [43]:
my_vault = Vault(os.getcwd())

In [ ]:
my_vault.extract_frontmatter()

In [38]:
def my_change(frontmatter, content):
    # if "authors" in frontmatter and "cedric" in frontmatter["authors"]:
    #     content.append("\n\n...cedric was here...")

    return (frontmatter, content)

In [39]:
my_vault.preview_change(my_change)

---
- 'type: note

  '
- 'authors:

  '
- '- cedric

  '
- '- "The Universe"

  '
- '- Keychron

  '
- 'created: "2024-10-17"

  '
---
# Hello World!

This is my story. It is short, and abru
